In [2]:
import pandas as pd
import time
import re
import os
import json

import openai
from dotenv import load_dotenv

In [ ]:
# file_path = "data/processed_data.csv"
# output_path = "data/processed_data.json"
# df = pd.read_csv(file_path)
# df.to_json(output_path, orient='records', force_ascii=False, indent=4)

In [ ]:
# file_path = "data/no_img_med_expert_sample500.csv"
# df = pd.read_csv(file_path)

# df['질문'] = df['제목'] + " " + df['질문'].fillna('')
# df = df[['질문', '답변']]

In [56]:
file_path = "data/med_expert.csv"
df = pd.read_csv(file_path, encoding='utf-8')

In [57]:
def extract_text_data(df):
    extracted_df = df[(df['photo_attached'] == '사진 없음') &
                     (df['answer_photo'] == '사진 없음') &
                     (df['video_attached'] == '동영상 없음') &
                     (df['answer_video'] == '동영상 없음')]
    
    print(f"원본 데이터 크기: {len(df)}")
    print(f"필터링 후 데이터 크기: {len(extracted_df)}")
    
    return extracted_df

df = extract_text_data(df)

원본 데이터 크기: 26090
필터링 후 데이터 크기: 16803


In [58]:
def remove_common_greetings(text):
    if not isinstance(text, str):
        return ''
    
    # 유니코드 특수문자(제로폭 공백 등) 제거
    text = re.sub(r'[\u200b\u200c\u200d\u2060\ufeff]', '', text)
    
    patterns = [
        # 시작 문구
        r'^안녕하세요.*?입니다\.?\s*',
        
        # 끝 문구
        r'\s*감사합니다\.?\s*$',
        r'\s*고맙습니다\.?\s*$',
        r'\s*안녕히\s*계세요\.?\s*$',
        r'\s*안녕히\s*가세요\.?\s*$',
        r'\s*수고하세요\.?\s*$',
    ]
    
    for pattern in patterns:
        text = re.sub(pattern, '', text, flags=re.MULTILINE | re.UNICODE).strip()
    
    return text

df['답변'] = df['답변'].apply(remove_common_greetings)

In [59]:
def prepare_qna_data(df):
    df['id'] = list(range(len(df)))
    df['question'] = df['제목'] + " " + df['질문'].fillna('')
    df['answer'] = df['답변']
    return df[['id', 'question', 'answer']]

df = prepare_qna_data(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16803 entries, 4 to 26089
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        16803 non-null  int64 
 1   question  16803 non-null  object
 2   answer    16803 non-null  object
dtypes: int64(1), object(2)
memory usage: 525.1+ KB


### LLM 기반 (gpt-4o-mini)

In [60]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI(api_key=OPENAI_API_KEY)

with open("prompt/filtering_prompt_system.txt", "r", encoding="utf-8") as file:
    system_prompt = file.read()
    
with open("prompt/filtering_prompt_user.txt", "r", encoding="utf-8") as file:
    base_user_prompt = file.read()

def filter_text(question, answer):
    question = str(question) if question is not None else ""
    answer = str(answer) if answer is not None else ""
    
    user_prompt = base_user_prompt.replace("{질문}", question).replace("{답변}", answer)
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}],
        seed=42
    )
    
    return response.choices[0].message.content.strip()

In [64]:
# 500개 랜덤 샘플링
sampled_df = df.sample(n=500, random_state=42).copy()

# 필터링된 결과를 저장할 리스트
filtered_questions = []
filtered_answers = []

start_time = time.time()
for _, row in sampled_df.iterrows():
    output = filter_text(row['question'], row['answer'])

    question_match = re.search(r"필터링된 질문:\s*(.+)", output)
    answer_match = re.search(r"필터링된 답변:\s*(.+)", output)

    filtered_question = question_match.group(1).strip() if question_match else ""
    filtered_question = "" if pd.isna(filtered_question) else filtered_question
    filtered_answer = answer_match.group(1).strip() if answer_match else ""
    filtered_answer = "" if pd.isna(filtered_answer) else filtered_answer
    
    filtered_questions.append(filtered_question)
    filtered_answers.append(filtered_answer)
    
sampled_df['filtered_question'] = filtered_questions
sampled_df['filtered_answer'] = filtered_answers

sampled_df['original_question'] = sampled_df['question']
sampled_df['original_answer'] = sampled_df['answer']
sampled_df = sampled_df[['id', 'original_question', 'filtered_question', 'original_answer', 'filtered_answer']]

output_file_csv = "data/filtered_data.csv"
sampled_df.to_csv(output_file_csv, index=False, encoding='utf-8')

output_file_json = "data/filtered_data.json"
sampled_df.to_json(output_file_json, orient='records', force_ascii=False, indent=4)

end_time = time.time()
elapsed_time = end_time - start_time

print(f"CSV 형식으로 '{output_file_csv}'에 저장되었습니다.")
print(f"JSON 형식으로 '{output_file_json}'에 저장되었습니다.")
print(f"전체 처리 시간: {elapsed_time:.2f}초")

CSV 형식으로 'data/filtered_data.csv'에 저장되었습니다.
JSON 형식으로 'data/filtered_data.json'에 저장되었습니다.
전체 처리 시간: 2044.16초


NaN 제거 및 column명 변경

In [66]:
file_path = "data/filtered_data.csv"
df = pd.read_csv(file_path, encoding='utf-8')
initial_row_count = len(df)
df = df[~(df['filtered_question'].isna()) & ~(df['filtered_answer'].isna())]
dropped_row_count = initial_row_count - len(df)
print(f"드롭될 행의 개수: {dropped_row_count}")

드롭될 행의 개수: 89


In [ ]:
file_path = "data/filtered_data.csv"
output_file_csv = "data/preprocessed_data.csv"
output_file_json = "data/preprocessed_data.json"

df = pd.read_csv(file_path)
initial_row_count = len(df)
df = df[~(df['filtered_question'].isna()) & ~(df['filtered_answer'].isna())]
dropped_row_count = initial_row_count - len(df)
df['question'] = df['filtered_question']
df['answer'] = df['filtered_answer']
df = df[['id', 'question', 'answer']]

df.to_csv(output_file_csv, index=False, encoding='utf-8')
df.to_json(output_file_json, orient='records', force_ascii=False, indent=4)

print(f"드롭된 행의 개수: {dropped_row_count}")
print(f"CSV 형식으로 '{output_file_csv}'에 저장되었습니다.")
print(f"JSON 형식으로 '{output_file_json}'에 저장되었습니다.")

드롭된 행의 개수: 89
CSV 형식으로 'data/preprocessed_data.csv'에 저장되었습니다.


중복되는 질문 제거

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import torch

def analyze_duplicate_questions(input_file, output_file, similarity_threshold=0.8):
    """
    유사한 질문들을 분석하고 시각화하는 함수
    
    Parameters:
    - input_file (str): 입력 CSV 파일 경로
    - output_file (str): 분석 결과 CSV 파일 경로
    - similarity_threshold (float): 유사도 임계값 (기본값: 0.8)
    
    Returns:
    - pandas.DataFrame: 유사한 질문 쌍 정보 데이터프레임
    """
    # 데이터 로드
    df = pd.read_csv(input_file)
    
    # Sentence BERT 모델 로드
    model = SentenceTransformer('jhgan/ko-sroberta-multitask')
    
    # 질문들에 대한 임베딩 생성
    question_embeddings = model.encode(df['question'].tolist(), convert_to_tensor=True)
    
    # 코사인 유사도 계산
    similarity_matrix = cosine_similarity(
        question_embeddings.cpu().numpy(), 
        question_embeddings.cpu().numpy()
    )
    
    # 유사한 질문 쌍 저장할 리스트
    similar_questions = []
    
    # 유사한 질문 쌍 찾기
    for i in range(len(df)):
        similar_indices = np.where(
            (similarity_matrix[i] > similarity_threshold) &  # 엄격하게 초과로 변경
            (similarity_matrix[i] < 1.0 - 1e-10)  # 완전히 동일한 행 제외
        )[0]
        
        for idx in similar_indices:
            if df.loc[i, 'question'] != df.loc[idx, 'question']:  # 텍스트 비교 추가
                similar_questions.append({
                    'original_question': df.loc[i, 'question'],
                    'similar_question': df.loc[idx, 'question'],
                    'similarity_score': float('{:.2f}'.format(similarity_matrix[i, idx]))
                })
    
    # 유사한 질문 쌍 데이터프레임 생성
    similar_questions_df = pd.DataFrame(similar_questions)
    
    # 중복 제거 (동일한 질문 쌍 제거)
    similar_questions_df = similar_questions_df.drop_duplicates(
        subset=['original_question', 'similar_question']
    )
    
    # 결과 파일 저장
    # similar_questions_df.to_csv(output_file, index=False, encoding='utf-8-sig')
    similar_questions_df.to_json(output_file.replace('.csv', '.json'), 
                                  orient='records', 
                                  force_ascii=False, 
                                  indent=4)
    
    # 유사한 질문 쌍 출력
    print("유사한 질문 쌍")
    print("------------------")
    for _, row in similar_questions_df.iterrows():
        print(f"\n원본 질문: {row['original_question']}")
        print(f"유사한 질문: {row['similar_question']}")
        print(f"유사도 점수: {row['similarity_score']:.2f}")
    
    # 통계 출력
    print(f"\n총 유사한 질문 쌍 수: {len(similar_questions_df)}")
    
    return similar_questions_df

# 사용 예시
input_path = "data/processed_data.csv"
output_path = "data/similar_questions.csv"

# 유사한 질문 분석
analyze_duplicate_questions(
    input_file=input_path, 
    output_file=output_path,
    # similarity_threshold=0.9
)

### 정규 표현식 검증

In [81]:
import re

def remove_common_greetings(text):
    if not isinstance(text, str):
        return ''

    # 유니코드 특수문자(제로폭 공백 등) 제거
    text = re.sub(r'[\u200b\u200c\u200d\u2060\ufeff]', '', text)
    
    patterns = [
        # 시작 문구
        r'^안녕하세요.*?입니다\.?\s*',
        
        # 끝 문구
        r'\s*감사합니다\.?\s*$',
        r'\s*고맙습니다\.?\s*$',
        r'\s*안녕히\s*계세요\.?\s*$',
        r'\s*안녕히\s*가세요\.?\s*$',
        r'\s*수고하세요\.?\s*$',
    ]
    
    for pattern in patterns:
        text = re.sub(pattern, '', text, flags=re.MULTILINE | re.UNICODE).strip()
    
    return text

text = "  감사합니다."

cleaned_text = remove_common_greetings(text)
print(f"original: {text}")
print(f"cleaned: {cleaned_text}")

original:   감사합니다.
cleaned: 


In [48]:
output = "필터링된 질문: NaN"
question_match = re.search(r"필터링된 질문:\s*(.+)", output)

filtered_question = question_match.group(1).strip()
filtered_question = "" if pd.isna(filtered_question) else filtered_question
    
print(filtered_question)

NaN


In [27]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI(api_key=OPENAI_API_KEY)

with open("prompt/filtering_prompt_2.txt", "r", encoding="utf-8") as file:
    base_prompt = file.read()

def filter_text(question, answer):
    question = str(question) if question is not None else ""
    answer = str(answer) if answer is not None else ""
    
    prompt = base_prompt.replace("{질문}", question).replace("{답변}", answer)
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "당신은 주어진 데이터를 필터링하는 전문가입니다."},
            {"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content.strip()

question = "강아지가 노란똥을 오늘 학교갔다와서 강아지랑 놀아주는데 애가 노란색의 되게 딱딱한 똥을 쌌더라구요...이건 무슨경우인가요? 빠른 답변 부탁드리겠습니다ㅜㅜ"
answer = "혹시 고구마 같은 것을 많이 먹이시지는 않으셨는지요? 설사이거나 매우 심각한 소화 불량질환을 겪은 경우가 아니라면 배변의 색이 변하는 경우는 특정 음식을 매우 많이 먹은 경우가 대부분 입니다"
output = filter_text(question, answer)

question_match = re.search(r"필터링된 질문:\s*(.+)", output)
answer_match = re.search(r"필터링된 답변:\s*(.+)", output)

filtered_question = question_match.group(1).strip() if question_match else ""
filtered_question = "" if pd.isna(filtered_question) else filtered_question
filtered_answer = answer_match.group(1).strip() if answer_match else ""
filtered_answer = "" if pd.isna(filtered_answer) else filtered_answer

print(f"original question: {question}")
print(f"filtered question: {filtered_question}")
print(f"original answer: {answer}")
print(f"filtered answer: {filtered_answer}")

original question: 강아지가 노란똥을 오늘 학교갔다와서 강아지랑 놀아주는데 애가 노란색의 되게 딱딱한 똥을 쌌더라구요...이건 무슨경우인가요? 빠른 답변 부탁드리겠습니다ㅜㅜ
filtered question: 강아지가 노란똥을 쌌습니다. 이는 무슨 경우인가요? 빠른 답변 부탁드립니다.
original answer: 혹시 고구마 같은 것을 많이 먹이시지는 않으셨는지요? 설사이거나 매우 심각한 소화 불량질환을 겪은 경우가 아니라면 배변의 색이 변하는 경우는 특정 음식을 매우 많이 먹은 경우가 대부분 입니다
filtered answer: 혹시 고구마 같은 것을 많이 먹이셨는지요? 설사이거나 매우 심각한 소화 불량 질환을 겪은 경우가 아니라면 배변의 색이 변하는 경우는 특정 음식을 매우 많이 먹은 경우가 대부분입니다.
